In [1]:

# !git clone https://github.com/ultralytics/yolov5
%cd /content/drive/MyDrive/yolov5
%pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output

/content/drive/MyDrive/yolov5
     |████████████████████████████████| 596 kB 4.3 MB/s 


In [2]:
import os, random
train_img_total, test_img_total, val_img_total = [], [], []
train_lbl_total, test_lbl_total, val_lbl_total = [], [], []
# 이미지 경로 입력(path)
path = "/content/drive/MyDrive/export/images/"
img_list = os.listdir(path)
img_list = [file for file in img_list if file.endswith(".jpg")]

# 클래스마다 이미지 분류
tan_0 = [x for x in img_list if "a7" in x or "a11" in x]
white_1 = [x for x in img_list if "a2" in x or "a4" in x or "a5" in x or "a8" in x or "a9" in x]
non_2 = [x for x in img_list if "b6" in x]
nop_3 = [x for x in img_list if "b7" in x]
nok_4 = [x for x in img_list if "b8" in x]
cal_5 = [x for x in img_list if "b3" in x]
gray_6 = [x for x in img_list if "a1" in x or "a6" in x]
nogyun_8 = [x for x in img_list if "a3" in x or "a12" in x]

# img_list_total에 전부 저장
img_list_total = [tan_0, white_1, non_2, nop_3, nok_4, cal_5, gray_6, nogyun_8]


In [ ]:
from glob import glob

In [ ]:
# 리스트 정리
from glob import glob
train_img_list = glob('/content/drive/MyDrive/yolov5/data/dis/train/images/*.jpg')

In [ ]:
with open("/content/drive/MyDrive/yolov5/data/dis/train.txt", "r") as f:
    train_img_list_2 = f.read()

In [ ]:
val_img_list = glob('/content/drive/MyDrive/yolov5/data/dis/val/images/*.jpg')

In [ ]:
len(train_img_list_2)

12431878

In [ ]:
len(val_img_list)

12725

In [ ]:
train_img_list

In [ ]:
import yaml
# txt 파일로 저장한다 !
with open('/content/drive/MyDrive/yolov5/data/dis/train.txt','w') as f:
  f.write('\n'.join(train_img_list) + '\n')
with open('/content/drive/MyDrive/yolov5/data/dis/val.txt','w') as f:
  f.write('\n'.join(val_img_list) + '\n')

NameError: ignored

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals())) #실제 셀의 포멧에서 파라메터 글로벌을 가져오고

In [ ]:
%%writetemplate /content/drive/MyDrive/yolov5/data/data.yaml

train: /content/drive/MyDrive/yolov5/data/dis/train/
val: /content/drive/MyDrive/yolov5/data/dis/val/

nc: 13
names: ["tenzer","white","N_nope","P_nope","K_nope","calshum_nope","gray_mold","firefurit","mozlalook","noguun","chuck","liilso","namhea"]
     

In [ ]:
# 파일을 열어본다

%cat /content/drive/MyDrive/yolov5/data/data.yaml


train: /content/drive/MyDrive/yolov5/data/dis/train/images
nc: 13
names: ["tenzer","white","N_nope","P_nope","K_nope","calshum_nope","gray_mold","firefurit","mozlalook","noguun","chuck","liilso","namhea"]
     

In [ ]:
# 모델 구성
import yaml

with open('/content/drive/MyDrive/yolov5/data/data.yaml', 'r') as stream :
  num_classes = str(yaml.safe_load(stream)['nc']) # nc : number classes

%cat /content/drive/MyDrive/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
%%writetemplate /content/drive/MyDrive/yolov5/models/custom_yolov5s.yaml
# nc 가 80인게 너무 큼 !
# nc 값에 변수로 지정해둔 것을 사용

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]  ]

In [ ]:
#원하는대로 저장이 됐는지 확인
%cat /content/drive/MyDrive/yolov5/models/custom_yolov5s.yaml


# nc 가 80인게 너무 큼 !
# nc 값에 변수로 지정해둔 것을 사용

# Parameters
nc: 13  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],

In [ ]:
pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Albumentations:
    # YOLOv5 Albumentations class (optional, used if package is installed)
    def __init__(self):
        self.transform = None
        try:
            import albumentations as A
            check_version(A.__version__, '1.0.3')  # version requirement

            self.transform = A.Compose([
                A.Blur(blur_limit=50, p=0.1),
                A.MedianBlur(blur_limit=51, p=0.1),
                A.ToGray(p=0.3)],
                bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

            logging.info(colorstr('albumentations: ') + ', '.join(f'{x}' for x in self.transform.transforms))
        except ImportError:  # package not installed, skip
            pass
        except Exception as e:
            logging.info(colorstr('albumentations: ') + f'{e}')

    def __call__(self, im, labels, p=1.0):
        if self.transform and random.random() < p:
            new = self.transform(image=im, bboxes=labels[:, 1:], class_labels=labels[:, 0])  # transformed
            im, labels = new['image'], np.array([[c, *b] for c, b in zip(new['class_labels'], new['bboxes'])])
        return im, labels

In [ ]:
# cache: 빠른 학습을 위한 이미지 캐시
%%time
%cd /content/drive/MyDrive/yolov5
!python train.py --img 640 --batch 32 --epochs 100 --data /content/drive/MyDrive/yolov5/data/data.yaml  --cfg /content/drive/MyDrive/yolov5/models/yolov5s.yaml --weights /content/drive/MyDrive/yolov5/yolov5s.pt --name dis_result --cache

/content/drive/MyDrive/yolov5
train: weights=/content/drive/MyDrive/yolov5/yolov5s.pt, cfg=/content/drive/MyDrive/yolov5/models/yolov5s.yaml, data=/content/drive/MyDrive/yolov5/data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=dis_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
YOLOv5 🚀 v6.1-359-g628c05c Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0

In [ ]:
!python val.py --weights runs/train/gochu_results21/weights/best.pt --data /content/drive/MyDrive/yolov5/data/data.yaml  --img 640 --iou 0.65 --half

In [ ]:
from google.colab import drive
drive.mount('/content/drive')